# Overview

this notebook runs the models and save the time and score
the saved dataframe can be found in `../data/processed/results_{date}`

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../")
from src.graph import NetworkGraph

In [ ]:
import time
import gurobi
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import random
import math
import datetime

In [ ]:
from src.models.DQL.DQL import run
from src.flows import *

In [ ]:
from flatland.envs.rail_env import RailEnv
from flatland.envs.observations import *
from flatland.envs.rail_generators import complex_rail_generator,rail_from_manual_specifications_generator,random_rail_generator, RailGenerator
from flatland.envs.schedule_generators import complex_schedule_generator, random_schedule_generator, ScheduleGenerator
from flatland.utils.rendertools import RenderTool
from flatland.envs.rail_env import RailEnv
from flatland.envs.observations import *
from flatland.envs.rail_generators import complex_rail_generator,rail_from_manual_specifications_generator,random_rail_generator, RailGenerator,sparse_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator, random_schedule_generator, ScheduleGenerator, sparse_schedule_generator
from flatland.utils.rendertools import RenderTool, AgentRenderVariant

### Helper functions

In [ ]:
def create_env(height,width,seed,number_of_agents=5):
    env = RailEnv(width=width,
              height=height,
              rail_generator=complex_rail_generator(nr_start_goal=20, nr_extra=1, 
                                                    min_dist=6, max_dist=99999, seed = seed),
              schedule_generator=complex_schedule_generator(),
              number_of_agents=number_of_agents)
    
    env.reset()
    
    return env

In [ ]:
def create_env_sparse(height,width,seed,number_of_agents=5):

    stochastic_data = {'prop_malfunction': 0,  # Percentage of defective agents
                       'malfunction_rate': 30,  # Rate of malfunction occurence
                       'min_duration': 3,  # Minimal duration of malfunction
                       'max_duration': 20  # Max duration of malfunction
                       }
    speed_ration_map = {1.: 1,  # Fast passenger train
                        1. / 2.: 0,  # Fast freight train
                        1. / 3.: 0,  # Slow commuter train
                        1. / 4.: 0}  # Slow freight train
    env = RailEnv(width=height,
                  height=width,
                  rail_generator=sparse_rail_generator(max_num_cities=number_of_agents+2,  
                                                       seed=seed, 
                                                       grid_mode=False,
                                                       max_rails_between_cities=4,
                                                       max_rails_in_city=4,
                                                       ),
                  schedule_generator=sparse_schedule_generator(speed_ration_map,seed = seed+2),
                  number_of_agents=number_of_agents,
                  stochastic_data=stochastic_data,  
                  obs_builder_object=GlobalObsForRailEnv(),
                  remove_agents_at_target=True
                  )
    env.reset()

    return env

In [ ]:
def run_flow_cg(env,*args):
    solver = Solver("cg_sparse.log",useDirections=True,verbose=False)
    cost =  solver.solve(env)
    number_of_variables = sum(solver.master.stats["variablesAdded"])
    time_init = solver.stats["timeInit"]
    return cost,number_of_variables, time_init

In [ ]:
def run_flow_af(env,*args):
    solver = Solver("cg_sparse.log",method="Arc Formulation",
                    useDirections=True,verbose=False)
    return solver.solve(env)

In [ ]:
def run_flow(env,height,width):
    
    #extract the transition matrix
    matrix_rail = np.array(env.rail.grid.tolist())
    
    #build the transition network
    flatlandNetwork = NetworkGraph(matrix_rail)

    #get the sources and sinks of the different agents
    sources = []
    sinks = []
    for agent in env.agents:
        sources.append(agent.initial_position)
        sinks.append(agent.target)
        
    #build the time exanded network and connect the sources and sinks
    TestNetworkTime = TimeNetwork(flatlandNetwork, depth=max(2*(height+width),40))
    TestNetworkTime.connect_sources_and_sink(sources,sinks)
    
    
    #get the Integral Program formulation of the problem and solve it 
    print(sources,sinks)
    mcflow = MCFlow(TestNetworkTime.graph,len(sources),TestNetworkTime.topology)
    mcflow.solve()
    
    #extract the path and compute the scores (total time spent by the agent in the grid)
    cost= 0
    if not mcflow.check_if_feasible():
        return "Infeasible"
    print(mcflow.m.objVal)
    paths = mcflow.extract_paths()
    for agent,path in paths.items():
        print(path)
        cost += -1+len(path)/2
       
    return cost

In [ ]:
def run_flow_rl(env,width,height,seed,n_start_goal=20,initial_value = 0,
             learning_rate = 0.8,gamma = 0.9,epsilon = 0.1,threshold = 0.3):
    number_agents = len(env.agents)
    n_episodes = width*height*number_agents*15
    n_steps = 2*(width+height)
    print(seed)
    cost, n_episodes, total_time = run(number_agents,
                                    width,height,
                                    n_start_goal,
                                    seed,
                                    n_episodes,
                                    n_steps,
                                    initial_value,
                                    learning_rate,
                                    gamma,
                                    epsilon,
                                    threshold)
    
    return cost, n_episodes, total_time

In [ ]:
def run_experiment_and_save_results(method, results,agents = 4, repetition = 5, create_env_f = create_env_sparse,
                                    pathToSavedGrids = None, grids_size = None,path= "../data/processed/"):
    nodes = []
    edges = []
    sizes_list = []
    if pathToSavedGrids is None:
        for sizes in tqdm(grids_size):
            for i in range(repetition):
                seed = sizes[0]*sizes[1]+i+10
                print(f'seed {seed}')
                env = create_env_f(sizes[0],sizes[1],seed,number_of_agents=agents)
                print("env created")
                
                start = time.time()
                cost,number_of_variables, time_init = run_flow_cg(env,sizes[0],sizes[1])
                sizes_list.append(sizes[0]*sizes[1])                    
                time_spent = time.time()-start
                results = results.append({'Size of the grid' : sizes[0]*sizes[1] , 
                                          'Score' : cost, "Time":time_spent,"Variables added":number_of_variables, 
                                          "time initial solution": time_init,
                                          "agents": agents,
                                          "Type":"Column Generation"} , 
                                         ignore_index=True)
                results.to_csv(path + "results_cg_sparse.csv")

                #start = time.time()
                #cost = run_flow_af(env,sizes[1],sizes[0],seed)
                #sizes_list.append(sizes[0]*sizes[1])                    
                #time_spent = time.time()-start
                #results = results.append({'Size of the grid' : sizes[0]*sizes[1] , 
                #                          'Score' : cost, "Time":time_spent, "Variables added": None,
                #                           "time initial solution": None,
                #                          "agents": agents,
                #                          "Type":"Arc Formulation"} , 
                #                         ignore_index=True)
                #results.to_csv(path + "results_af.csv")
                print("---")
                                
    else:
        raise NotImplementedError("Load the map and proceed to inference")

### Constant definition

In [ ]:
grids_size = [(25,25),(30,30),(40,40),(50,50),(60,60),(70,70),(80,80)]

In [ ]:
results = pd.read_csv("../data/processed/cg/results_cg_sparse.csv", index_col = 'Unnamed: 0')

In [ ]:
results

In [ ]:
test  = results[results.agents == 5]

In [ ]:
mean = test.groupby(['Size of the grid']).mean()

In [ ]:
mean

In [ ]:
df = results

In [ ]:
(100*mean["time initial solution"].values/mean["Time"].values).mean()

## Actual run

In [ ]:
for number in [5,10,20,30]:
    results = pd.read_csv("../data/processed/cg/results_cg_sparse.csv", index_col = 'Unnamed: 0')
        run_experiment_and_save_results(run_flow, results,
                                    agents = number, 
                                    repetition = 5, 
                                    grids_size = grids_size,
                                    path = "../data/processed/cg/")

# PLOTS

In [ ]:
df = pd.read_csv("../data/processed/small_grids/results_comparison5.csv", index_col = 'Unnamed: 0')
df2 = pd.read_csv("../data/processed/small_grids/results_comparison10.csv", index_col = 'Unnamed: 0')

In [ ]:
df = df2

In [ ]:
plt.rcParams['savefig.facecolor'] = (46/255., 48/255., 55/255.)
plt.rcParams['axes.facecolor'] = (46/255., 48/255., 55/255.)
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.spines['bottom'].set_color("white")
ax.spines['top'].set_color("white") 
ax.spines['right'].set_color('white')
ax.spines['left'].set_color('white')
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
ax.title.set_color('white')
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')
fig.set_facecolor((46/255., 48/255., 55/255.))
ax.set_facecolor((46/255., 48/255., 55/255.))


plt.scatter(df[df.agents == 5]['Size of the grid'].values,
            df[df.agents == 5].Time/3600,label = "5 agents")
plt.scatter(df[df.agents == 10]['Size of the grid'].values,
            df[df.agents == 10].Time/3600, label = "10 agents")
plt.scatter(df[df.agents == 20]['Size of the grid'].values,
            df[df.agents == 20].Time/3600, label = "20 agents")
#sizes_jitter = [np.random.normal(x, 2,1) for x in sizes]
#plt.scatter(sizes_jitter,diff)
plt.xlabel("size of the grid in cell number")

#x1 = [15*15,20*20,25*25]
#squad = ['(15,15)','(20x20)','(25,25)']#,'(20,20)','(25,20)','(25,25)','(25,30)']

#ax.set_xticks(x1)
#ax.set_xticklabels(squad)

plt.ylabel("Time to solution in minutes")

l = plt.legend()
for text in l.get_texts():
    text.set_color("white")



ax = plt.gca()
plt.tight_layout()
plt.savefig("../data/processed/comparison_cg_time.png", dpi = 300)

In [ ]:
df_20_agents = pd.read_csv("../data/processed/results_20_agents.csv",index_col = 'Unnamed: 0')

In [ ]:
df_10_agents = pd.read_csv("../data/processed/results_10_agents.csv",index_col = 'Unnamed: 0')

In [ ]:
df_4_agents = pd.read_csv("../data/processed/results_4_agents.csv",index_col = 'Unnamed: 0')

In [ ]:
plt.rcParams['savefig.facecolor'] = (46/255., 48/255., 55/255.)
plt.rcParams['axes.facecolor'] = (46/255., 48/255., 55/255.)
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.spines['bottom'].set_color("white")
ax.spines['top'].set_color("white") 
ax.spines['right'].set_color('white')
ax.spines['left'].set_color('white')
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
ax.title.set_color('white')
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')
fig.set_facecolor((46/255., 48/255., 55/255.))
ax.set_facecolor((46/255., 48/255., 55/255.))


plt.scatter(df_4_agents['Size of the grid'],df_4_agents['Time'].values/60, label = "4 trains",alpha= 0.8)
plt.scatter(df_10_agents['Size of the grid'],df_10_agents['Time'].values/60, label = "10 trains",alpha = 0.8)
plt.scatter(df_20_agents['Size of the grid'],df_20_agents['Time'].values/60, label = "20 trains",alpha = 0.8)


plt.xlabel("size of the grid (number of cells)")

x1 = [20**2,40**2,50*50,60*60]
squad = ['(20x20)','(40x40)','(50,50)','(60x60)']

ax.set_xticks(x1)
ax.set_xticklabels(squad)

plt.ylabel("time until solution (min)")
#plt.title("Experiment of multicommodity flow formulation until memory error (model > 40 Gb)")

l = plt.legend()
for text in l.get_texts():
    text.set_color("white")



ax = plt.gca()
plt.tight_layout()
plt.savefig("../data/processed/time.png", dpi = 300)